In [66]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
import pandas as pd
import time
import re

In [67]:
df=pd.read_csv('inmates.csv')

In [68]:
df.head()

,Unnamed: 0,admitted_dt,age,bradh,custody_level,gender,infraction,inmate_status_code,inmateid,race,sealed,srg_flg,top_charge
0,0,2018-06-01T23:39:50.000,30.0,N,MED,M,N,DPV,20025,B,N,N,NaN
1,1,2018-06-22T16:56:05.000,33.0,N,MAX,M,N,DE,58432,B,N,N,160.10
2,2,2018-06-22T17:17:42.000,64.0,Y,MIN,M,N,DE,20196082,A,N,N,130.55
3,3,2018-06-01T16:37:28.000,39.0,N,MIN,M,N,CS,10013090,B,N,N,110-220.39
4,4,2017-09-30T16:09:23.000,21.0,Y,MED,M,Y,DE,20187487,B,N,N,130.50


In [69]:
df['clean_charge']=df.top_charge.fillna(0)

In [71]:
df['charges']=df.clean_charge.str.extract(r"(\d\d\d[.]\d\d)").fillna(0)

In [72]:
df['additional_charges']=df.clean_charge.str.extract(r"(\d\d\d)-").fillna(0)

In [73]:
top_charges=df.charges.value_counts().head(41)

In [74]:
top_charges=top_charges[1:]

In [75]:
top_charges=top_charges.reset_index()

In [76]:
top_charges.index.rename('charges')

RangeIndex(start=0, stop=40, step=1, name='charges')

In [77]:
top_charges.columns=['charges', 'value_counts']

In [78]:
driver = webdriver.Chrome()

In [79]:

def penal_code(row):
    penal=[]
    try:
        while True:
            
            driver.get('https://codes.findlaw.com/ny/penal-law/')
            print('going to website')

            driver.find_element_by_id('codesearchbar').send_keys("pen "+row['charges'])
            print('sending the codes')

            button = driver.find_element_by_class_name('sc_search_btn')
            driver.execute_script("arguments[0].scrollIntoView(true)", button)
            button.click()
            print('searching')

            time.sleep(2)

            button = driver.find_element_by_id('gsa_n_1')
            driver.execute_script("arguments[0].scrollIntoView(true)", button)
            button.click()
            print('looking for penal code')
            
            time.sleep(2)
            print('after sleep')

            search_button = driver.find_element_by_id('gsa_link_1')
            search_button.click()

            results=driver.find_element_by_class_name('baseIncluder')
            
            
            print('writing results')
            print('................')
            return results.text
            

    except:
        print('problem')

In [80]:
top_charges['code_charges']=top_charges.apply(penal_code, axis=1)

going to website
sending the codes
searching
looking for penal code
after sleep
writing results
................
going to website
sending the codes
searching
looking for penal code
after sleep
writing results
................
going to website
sending the codes
searching
looking for penal code
after sleep
writing results
................
going to website
sending the codes
searching
looking for penal code
after sleep
writing results
................
going to website
sending the codes
searching
looking for penal code
after sleep
writing results
................
going to website
sending the codes
searching
looking for penal code
after sleep
writing results
................
going to website
sending the codes
searching
looking for penal code
after sleep
writing results
................
going to website
sending the codes
searching
looking for penal code
after sleep
writing results
................
going to website
sending the codes
searching
looking for penal code
after sleep
writing results


In [81]:
pd.set_option('display.max_colwidth', -1)

top_charges.to_csv('charges_test.csv', index=False)

In [37]:
#top_charges['clean_top_charges']=top_charges.code_charges.str.extract(r"\d\d\d[.]\d\d-?\w? (.*)")

In [65]:
#new_df=df.merge(top_charges, on='charges', how='outer')
new_df=df.join(top_charges.set_index('charges'), on='charges')
df['admitted']=pd.to_datetime(df.admitted_dt, format='%Y/%m/%d %H:%M:%S')
new_df.to_csv('inmates_penal_code.csv', index=False)

In [82]:
top_charges

,charges,value_counts,code_charges
0,125.25,858,"New York Consolidated Laws, Penal Law - PEN § 125.25 Murder in the second degree"
1,160.15,442,"New York Consolidated Laws, Penal Law - PEN § 160.15 Robbery in the first degree"
2,120.05,364,"New York Consolidated Laws, Penal Law - PEN § 120.05 Assault in the second degree"
3,265.03,353,"New York Consolidated Laws, Penal Law - PEN § 265.03 Criminal possession of a weapon in the second degree"
4,140.25,329,"New York Consolidated Laws, Penal Law - PEN § 140.25 Burglary in the second degree"
5,220.39,314,"New York Consolidated Laws, Penal Law - PEN § 220.39 Criminal sale of a controlled substance in the third degree"
6,220.16,310,"New York Consolidated Laws, Penal Law - PEN § 220.16 Criminal possession of a controlled substance in the third degree"
7,160.10,304,"New York Consolidated Laws, Penal Law - PEN § 160.10 Robbery in the second degree"
8,140.20,263,"New York Consolidated Laws, Penal Law - PEN § 140.20 Burglary in the third degree"
9,155.25,224,"New York Consolidated Laws, Penal Law - PEN § 155.25 Petit larceny"
